In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from datetime import datetime

In [2]:
DATA_DIR = "data"

In [3]:
#Extracting intericite data
times = pd.read_csv(DATA_DIR + "/export-intercites-gtfs-last/stop_times.txt")
stops = pd.read_csv(DATA_DIR + "/export-intercites-gtfs-last/stops.txt")
#Extracting ter data
stops_ter = pd.read_csv(DATA_DIR + "/ter/stops.txt")
stop_times_ter = pd.read_csv(DATA_DIR + "/ter/stop_times.txt")

In [4]:
#fix times which hours exceeds 24
def fix_times(time) : 
    
    if int(time.split(":")[0]) >= 24:
        return "{}:".format(int(time.split(":")[0]) - 24)+":".join(time.split(":")[1:])
    return time

def compute_duration(arrival_list, departure_list) : 
    durations = []
    for i in range(len(arrival_list) -1) : 
        durations.append((datetime.strptime(departure_list[i+1],"%H:%M:%S") - datetime.strptime(arrival_list[i],"%H:%M:%S")).seconds / 60)
    return durations

In [5]:
# fixing time for intercite
times["arrival_time"] = times["arrival_time"].apply(fix_times)
times["departure_time"] = times["departure_time"].apply(fix_times)
#fix time for ter
stop_times_ter["arrival_time"] = stop_times_ter["arrival_time"].apply(fix_times)
stop_times_ter["departure_time"] = stop_times_ter["departure_time"].apply(fix_times)

In [6]:
cols_1 = times.columns
cols_2 = stops.columns 

In [7]:
table = pd.merge(times, stops, how = "left", left_on = times.stop_id, right_on= stops.stop_id).rename(columns = {"key_0" : "stop_id"})[[cols_1[0] , cols_1[1], cols_1[2] , cols_2[0],cols_2[1],cols_2[3],cols_2[4]]]
timetable = table.groupby("trip_id").agg(list).reset_index()

timetable["durations"] = timetable[["arrival_time","departure_time"]].apply(
    lambda x : compute_duration(
        x["arrival_time"], x["departure_time"]), axis = 1)

timetable.to_csv("./data/export-intercites-gtfs-last/timetable_intercity.csv")


In [8]:
timetable.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_name,stop_lat,stop_lon,durations
0,OCESN001001F0100459990,"[08:21:00, 11:06:00, 11:26:00, 11:52:00, 12:17...","[08:21:00, 11:09:00, 11:29:00, 12:00:00, 12:20...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Gare de Paris-Est, Gare de Nancy-Ville, Gare ...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[168.0, 23.0, 34.0, 28.0, 33.0]"
1,OCESN001001F0100659999,"[07:06:00, 10:26:00, 10:50:00, 11:14:00, 11:31...","[07:06:00, 10:32:00, 10:52:00, 11:16:00, 11:33...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Gare de Paris-Est, Gare de Nancy-Ville, Gare ...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[206.0, 26.0, 26.0, 19.0, 33.0]"
2,OCESN001001F0100759984,"[08:21:00, 11:06:00, 11:26:00, 11:52:00, 12:17...","[08:21:00, 11:09:00, 11:29:00, 12:00:00, 12:20...","[StopPoint:OCECorail Intercité-87113001, StopP...","[Gare de Paris-Est, Gare de Nancy-Ville, Gare ...","[48.87656977, 48.68978225, 48.58799369, 48.737...","[2.35915061, 6.17427169, 6.49703457, 7.0527805...","[168.0, 23.0, 34.0, 28.0, 35.0]"


In [9]:
merged_ter = pd.merge(stop_times_ter,stops_ter,how='left', left_on=stop_times_ter.stop_id,right_on = stops_ter.stop_id) \
            [['trip_id','arrival_time','departure_time','stop_name','stop_lat','stop_lon']]

timetable_ter = merged_ter.groupby("trip_id").agg(list).reset_index()

timetable_ter["durations"] = timetable_ter[["arrival_time","departure_time"]].apply(
    lambda x : compute_duration(
        x["arrival_time"], x["departure_time"]), axis = 1)

In [10]:
timetable_ter.head(3)

,trip_id,arrival_time,departure_time,stop_name,stop_lat,stop_lon,durations
0,OCEDB081400F0100134686,"[05:26:00, 05:45:00, 06:06:00, 06:28:00]","[05:26:00, 05:53:00, 06:07:00, 06:28:00]","[Gare de Wissembourg, Gare de WINDEN PFALZ, Ga...","[49.03153308, 49.09544004, 49.19828733, 49.350...","[7.95030589, 8.1224056, 8.12533019, 8.14067588]","[27.0, 22.0, 22.0]"
1,OCEDB081400F0200234682,"[05:26:00, 05:45:00, 06:06:00, 06:28:00]","[05:26:00, 05:53:00, 06:07:00, 06:28:00]","[Gare de Wissembourg, Gare de WINDEN PFALZ, Ga...","[49.03153308, 49.09544004, 49.19828733, 49.350...","[7.95030589, 8.1224056, 8.12533019, 8.14067588]","[27.0, 22.0, 22.0]"
2,OCEDB081401F0100134545,"[05:05:00, 05:21:00]","[05:05:00, 05:21:00]","[Gare de WINDEN PFALZ, Gare de Wissembourg]","[49.09544004, 49.03153308]","[8.1224056, 7.95030589]",[16.0]


In [11]:
timetable_ter.to_csv("./data/ter/timetable_ter.csv")
